In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
!ls 
os.chdir('gdrive/My Drive/dl20/final_project/FinalProject/fastRCNN')

!ls

gdrive	sample_data
data_helper.py	      objectDectionFastRCNN_042020.ipynb   __pycache__
helper.py	      objectDectionFastRCNN_042720.ipynb   run_test.py
model_loader.py       objectDectionFastRCNN_v2.ipynb	   transforms.py
nhung_data_helper.py  objectDectionFastRCNN_v2_TEST.ipynb  utils.py


In [0]:
import sys
import os
#os.chdir('/scratch/nhl256/dl_project/code/')

import random
import argparse

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import LabeledDataset
from helper import *

import math
import time

import utils

In [10]:
sys.path.append('gdrive/My Drive/dl20/final_project/FinalProject/data/labeled')
sys.path.append('gdrive/My Drive/dl20/final_project/FinalProject/model')

image_folder = '../data/labeled'
annotation_csv = '../data/annotation.csv'
object_detection_model_path = '../model/object_detection_resnet18.pth'

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

cuda:0


In [0]:
def get_transform(): 
    return torchvision.transforms.ToTensor()

In [0]:
labeled_scene_index = np.arange(107, 108)
labeled_trainset = LabeledDataset(
    image_folder=image_folder,
    annotation_file=annotation_csv,
    scene_index=labeled_scene_index,
    transform=get_transform(),
    extra_info=False
    )
dataloader = torch.utils.data.DataLoader(
    labeled_trainset,
    batch_size=1,
    shuffle=False,
    num_workers=4
    )
trainloader = torch.utils.data.DataLoader(labeled_trainset,
                                          batch_size=2,
                                          shuffle=True,
                                          num_workers=2,
                                          collate_fn=collate_fn)

## model_loader.py

### For Object Detection

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import math
import time

import utils

In [0]:
def get_transform(): 
    return torchvision.transforms.ToTensor()


def extract_features(one_sample):
    feature_extractor = torchvision.models.resnet18(pretrained=False)
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
    feature_extractor.cuda()
    return feature_extractor(one_sample)

def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

def prepare_inputs(samples):
    """
    Input: samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    Output: a list of batch_size tensor, each tensor with size [512, 16, 114]
    """
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = samples[i]
        features = extract_features(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 512, 160)
        #print(features.shape)
        fe_batch.append(features)
    
    return fe_batch

# Need to convert this to 
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)


In [0]:
class ModelLoader():
    # Fill the information for your team
    team_name = 'team_name'
    round_number = 1
    team_member = []
    contact_email = '@nyu.edu'

    def __init__(self, model_file='put_your_model_file_name_here'):
        """
        model_file = {'get_bboxes_model': object_detection_model_path,
                      'get_binary_RM_model': None}
        """
        self.bboxes_model_weights =  model_file['get_bboxes_model']
        self.bboxes_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        self.bboxes_model = self.bboxes_model.to(device)
        #self.model_path = '/scratch/nhl256/dl_project/model/object_detection_resnet18_epoch2.pth'
        self.bboxes_model.load_state_dict(torch.load(self.bboxes_model_weights))
        self.bboxes_model.eval()

    def get_bounding_boxes(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
        # where N is the number of object
        inputs = prepare_inputs(samples)
        predictions = self.bboxes_model(inputs)
        res = []
        for i in range(len(predictions)):
            prediction = predictions[i]
            pred_bboxes = prediction['boxes']
            reorder_pred_bboxes = reorder_coord(pred_bboxes)
            res.append(reorder_pred_bboxes)

        return res

    def get_binary_road_map(self, samples):
        # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
        # You need to return a cuda tensor with size [batch_size, 800, 800] 
        
        return torch.rand(1, 800, 800) > 0.5

## Get Bounding Boxes Threat Score

In [0]:
model_file = {'get_bboxes_model': object_detection_model_path,
                      'get_binary_RM_model': None}

In [0]:
model_loader = ModelLoader(model_file)

In [16]:
print('number of samples in dataloader is {}'.format(dataloader.__len__()))

number of samples in dataloader is 126


In [21]:
total = 0
total_ats_bounding_boxes = 0
total_ts_road_map = 0
for i, data in enumerate(dataloader):
    total += 1
    sample, target, road_image = data
    sample = sample.cuda()

    predicted_bounding_boxes = model_loader.get_bounding_boxes(sample)[0].cpu()
    

    ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                    target['bounding_box'][0])
    print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
    print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

    total_ats_bounding_boxes += ats_bounding_boxes

Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([68, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([65, 2, 4])
ats_bounding_boxes 0.010049150325357914
Number of pred bboxes torch.Size([59, 2, 4])
ats_bounding_boxes 0.011021647602319717
Number of pred bboxes torch.Size([69, 2, 4])
ats_bounding_boxes 0.009490863420069218
Number of pred bboxes torch.Size([70, 2, 4])
ats_bounding_boxes 0.006736313924193382
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.004126581363379955
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.0040640574879944324
Number of pred bboxes torch.Size([62, 2, 4])
ats_bounding_boxes 0.007450771518051624
Number of pred bboxes torch.Size([63, 2, 4])
ats_bounding_boxes 0.004003399517387152
Number of pred bboxes torch.Size([60, 2, 4])
ats_bounding_boxes 0.0
Number of pred bboxes torch.Size([73, 2, 4])
at

In [22]:
total_ats_bounding_boxes

tensor(0.0924)